In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import train_test_split
import time

In [2]:
data = pd.read_csv("iris.data", names = ["f1","f2","f3","f4","class"])
Y = data["class"]
X = data.drop(columns=["class"])
X = np.asarray(X)
Y = np.asarray(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
num_class = len(np.unique(Y))
classes = dict()
cnt = 0
for index,value in enumerate(np.unique(Y)):
    classes[value] = cnt
    cnt+=1

In [3]:
def accuracy(y_pred, y_actual,classes):
    right = 0
    wrong = 0
    for index in range(len(y_pred)):
        if (y_pred[index] == classes[y_actual[index]]):
            right+=1
        else:
            wrong+=1

    return (right  / (right + wrong))

def convert_labels( Y):
        res = np.zeros(shape = (Y.shape[0],num_class))
        for i, value in enumerate(Y): res[i][classes[value]] = 1

        return res

In [4]:
tmpY_train = convert_labels(Y_train)
tmpY_test = convert_labels(Y_test)

In [5]:
class PythonLogisticReg():
    def __init__(self, lr, iters, num_class, classes):
        self.lr = lr
        self.num_class = num_class
        self.iters = iters
        self.classes = classes
    
    def softmax_s(self, input):
        temp = [math.exp(v) for v in input]
        total = sum(temp)
        return [t / total for t in temp]

    def softmax(self,Z):
        res = []
        for i in range(len(Z)):
            res.append(self.softmax_s(Z[i]))

        return res
        
    def dot(self, X, Y):
        res = self.init_matrix(len(X),len(Y[0]))
        for i in range(len(X)):
            for j in range(len(Y[0])):
                for k in range(len(Y)):
                    res[i][j] += X[i][k] * Y[k][j]
        return res
    def argmax(self,X):
        res = [0 for i in range(len(X))]
        for i in range(len(X)):
            max_v = -100000000
            max_i = 0
            for j in range(len(X[0])):
                if X[i][j] > max_v:
                    max_v = X[i][j]
                    max_i = j
            res[i] = max_i
        return res
    
    def init_matrix(self, n, m):
        res = []
        for i in range(n):
            tmp = []
            for j in range(m):
                tmp.append(0)
            res.append(tmp)
        return res
    
    def convert_labels(self, Y):
        res = self.init_matrix(len(Y), self.num_class)

        for i in range(len(Y)):
            index = self.classes[Y[i]]
            res[i][index] = 1

        return res
    
    def gradients(self,X,Y):
        return self.dot(X,Y)
    
    def subtract(self, X, Y):
        for i in range(len(X)):
            for j in range(len(X[0])):
                X[i][j]-=Y[i][j]
        return X
    
    def transpose(self,X):
        return [[X[j][i] for j in range(len(X))] for i in range(len(X[0]))]
    
    def fit(self,X,Y):
        self.X = X
        self.n = len(X)
        self.m = len(X[0])
        self.Y = self.convert_labels(Y)
        self.W = self.init_matrix(self.m, self.num_class)

        for _ in range(self.iters):
            
            y_pred = self.softmax(self.dot(X,self.W))
            
            gradient = self.gradients(self.transpose(X), self.subtract(y_pred,self.Y))

            for i in range(len(gradient)):
                for j in range(len(gradient[0])):
                    gradient[i][j] *= self.lr
            self.W  = self.subtract(self.W,gradient)

        return self
    
    def predict(self,X):
        probas = self.softmax(self.dot(X, self.W))
        
        return self.argmax(probas)

In [6]:
class NumpyLogisticReg():
    def __init__(self, lr, iters, num_class, classes):
        self.lr = lr
        self.num_class = num_class
        self.iters = iters
        self.classes = classes
        
    def softmax(self,Z):
        e_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True)) 

        return e_Z / np.sum(e_Z, axis=1, keepdims=True)

    def convert_labels(self, Y):
        res = np.zeros(shape = (Y.shape[0],self.num_class))
        for i, value in enumerate(Y): res[i][self.classes[value]] = 1

        return res

    def fit(self,X,Y):
        self.m , self.n =  X.shape
        self.X = X
        self.Y = self.convert_labels(Y)
        
        self.W = np.zeros(shape = (self.n, self.num_class))

        for _ in range(self.iters):
            y_preds = self.softmax(self.X.dot(self.W))
            
            gradients = self.X.T.dot(y_preds - self.Y)

            self.W -= self.lr * gradients

        return self
    
    def predict(self, X):

        probas = self.softmax(X.dot(self.W))

        return np.argmax(probas, axis=1)
        

In [7]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Input
from keras.optimizers import Adam

input = Input(shape = (X_train.shape[1],))
output = Dense(3, activation = "softmax")(input)

Krmodel = Model(inputs = input, outputs = output)

opt = Adam(learning_rate = 0.001)
Krmodel.compile(loss = "binary_crossentropy", optimizer=opt, metrics = ["accuracy"])
Krmodel.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 3)                 15        
                                                                 
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


2023-03-09 07:22:39.167719: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-09 07:22:39.167825: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
print("Train")
start_time = time.time()
LR = NumpyLogisticReg(lr = 0.001, iters = 500, num_class = num_class,classes = classes)
LR.fit(X_train,Y_train)
print("--- %s seconds ---" % (time.time() - start_time))
Y_pred = NumpyLogisticReg.predict(X_train)
acc = accuracy(Y_pred, Y_train, classes)
print(acc)

print("Test")
start_time = time.time()
Y_pred = LR.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

acc = accuracy(Y_pred, Y_test, classes)
print(acc)

Train
--- 0.009154081344604492 seconds ---


TypeError: predict() missing 1 required positional argument: 'X'

In [ ]:
print("Train")
start_time = time.time()
PythonLG = PythonLogisticReg(0.001, 500, num_class, classes)
PythonLG.fit(X_train, Y_train)

print("--- %s seconds ---" % (time.time() - start_time))
Y_pred = PythonLG.predict(X_train)
acc = accuracy(Y_pred, Y_train, classes)
print(acc)
print("Test")
start_time = time.time()
Y_pred = PythonLG.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

acc = accuracy(Y_pred, Y_test, classes)
print(acc)

Train
--- 0.39046502113342285 seconds ---
0.9833333333333333
Test
--- 0.00014781951904296875 seconds ---
0.9333333333333333


In [ ]:
print("Train")
start_time = time.time()
Krmodel.fit(X_train,tmpY_train, epochs = 500,verbose = False)
print(Krmodel.evaluate(X_train, tmpY_train)[1])
print("--- %s seconds ---" % (time.time() - start_time))

print("Test")
start_time = time.time()
print(Krmodel.evaluate(X_test, tmpY_test)[1])
print("--- %s seconds ---" % (time.time() - start_time))


Train


2023-03-09 07:22:14.779453: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-09 07:22:14.903793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 0s 10ms/step - loss: 0.3239 - accuracy: 0.8583
0.8583333492279053
--- 10.316765069961548 seconds ---
Test
1/1 [==============================] - 0s 38ms/step - loss: 0.3121 - accuracy: 0.7667
0.7666667103767395
--- 0.052453041076660156 seconds ---


2023-03-09 07:22:24.990143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
learning_rate = 0.001
num_epochs = 500

with tf.name_scope("Declaring_placeholder"):
    X = tf.placeholder(tf.float32, [None, 4])
    y = tf.placeholder(tf.float32, [None, 3])
    
with tf.name_scope("Declaring_variables"):
    W = tf.Variable(tf.zeros([4, 3]))
    b = tf.Variable(tf.zeros([3]))
    
with tf.name_scope("Declaring_functions"):
    y_ = tf.nn.softmax(tf.add(tf.matmul(X, W), b))

with tf.name_scope("calculating_cost"):
    cost = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_)
with tf.name_scope("declaring_gradient_descent"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

print("Train")
start_time = time.time()
with tf.name_scope("starting_tensorflow_session"):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(num_epochs):
            cost_in_each_epoch = 0
            _, c = sess.run([optimizer, cost], feed_dict={X: X_train, y: tmpY_train})
            cost_in_each_epoch += c

        correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("Accuracy:", accuracy.eval({X: X_train, y: tmpY_train}))
        print("--- %s seconds ---" % (time.time() - start_time))

        print("Test")
        start_time = time.time()
        print("Accuracy:", accuracy.eval({X: X_test, y: tmpY_test}))
        print("--- %s seconds ---" % (time.time() - start_time))

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Train


2023-03-09 07:22:25.195787: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-09 07:22:25.195807: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-09 07:22:25.196559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-03-09 07:22:25.197226: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-09 07:22:25.200200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy: 0.975
--- 2.2629659175872803 seconds ---
Test
Accuracy: 0.9333334
--- 0.007632017135620117 seconds ---


2023-03-09 07:22:27.445691: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
